## hadoop
#### 起源
google搜索引擎
* google集装箱数据中心，每个集装箱里又1160台服务器，标准化。能效比1.25.一般公司在2.0
* 解决问题：
    * 怎么存储大量网页，数据冗余，保证安全
    * 搜索算法，如果要模糊匹配 LIKE xxx, 在SQL中是不能用索引的，而是要全表扫描，慢（用倒排索引）
    * page-rank计算问题

#### 倒排索引
```
word       index
Google    {1:1}, {3:2}      // documentId: offset
```
* 分词 -> 去掉stopword -> 倒排索引

##### 分词
* 一种办法，用字典。 切出一个词，在字典里查看有没有

#### page-rank
* 垃圾里面找黄金
* 页面的重要性如何判断？
    * 点击量 -- google爬虫爬不到
    * 通过被引用数量判断，如果被引用得多就证明该页面重要
    * 被page-rank高的页面指向，该页面重要性更高
* google matrix：

```

1 -> 2
|  x |
3 -> 4

0    0    0  0
1/3  0    0  1
1/3  1/2  1  0
1/3  1/2  0  0

G = aS + (1-a)1/n * U
q=Gq, 求上特征矩阵的特征向量。特征向量q[i]就是rank-score
```
通过不断的map-reduce，来进行矩阵的运算，最后出一个千上万列的matrix的特征向量

#### Google tech key
* GFS
* Map-reduce
* Big table

#### lucene
* Hadoop的源起 -- Lucene
* java 做的全文检索工具
* lucene的微缩版：Nutch

### Hadoop子项目
* HBASE, PIG, HIVE
* MapReduce, HDFS, ZooKeeper
* Core, Avro

### 架构
* JobTracker, TaskTracker
* NameNode, DataNode

##### HDFS
* NameNode - Master
   * HDFS的守护程序
   * 作为分布式文件系统的总控作用
   * 对内存和I/O进行集中管理
   * 是单点，有单点故障的风险
* Secondary NameNode 辅助名称节点
   * 作为NameNode的备份
   * 与NameNode通讯，对NameNode快照
* DataNode - Slave
   * 数据分布式存储

##### MapReduce
* JobTracker - Master
   * 用于处理作业(用户提交代码)的后台程序
   * 决定那些文件参与处理，然后切割task并分配节点
   * 监控task，重启失败的task
   * 每个集群只有唯一的一个JobTracker，位于master节点
* TaskTrakcer - Slave
   * 位于Slave节点和与datanode结合
   * 管理个子节点的task，由jobTracker分配
   * 每个节点自会有一个taskTracker，可以启动多个JVM

#### 安装使用
skip
##### 测试

```
> bin/hadoop dfs -put ../intput in 
> bin/hadoop dfs -ls ./in/*           // hadoop 分布式文件操作命令
```

```
> bin/hadoop jar hadoop-0.20.2-example.jar wordcount in out     //运行一个叫wordcount的作业
```

```
> bin/hadoop dfs -ls ./out
  /user/grid/out/_logs
  /user/grid/out/part-r-00000
```

* 通过浏览器检查jobtracker在结点50030端口，监控jobtracker
   * http://localhost:50030/jobtracker.jsp
   * list: complete jobs/filed jobs/running jobs/local logs
   * job details: file link, running time, etc
* 通过浏览器访问namenode所在结点在50070端口监控集群
   * browse filesystem
   * cluster summary
   * namenode storage
   * logs

## HDFS
#### 物理存储
* 每个服务器里有路径
   * blk—34820934092830
   * blk—34820934092830_1037.meta

#### 设计思想
* 硬件错误是常态性的，需要冗余
* 流式数据方位，支持数据的批量读取，而**非随机读取**，hadoop擅长**数据分析**而**不**是**事务处理**。NOT OLTP, OLAP
* 简单的一致性模型，为了降低系统复杂性，对文件采取一次性写，多次读。文件一经写入，无法修改
* 程序采用“数据就进”的原则分配节点执行

#### 体系结构
* Namenode
   * 记录文件数据块在哪个datanode的位置和副本信息
   * 元数据操作：事务日志
   * 元数据操作：映射文件
* Datanode
   * 一次写入，多次读取，不修改
   * 文件由数据块组成，一般64MB
* 事务日志
* 映像文件
* Secondary Namenode

#### 可靠性
* 冗余副本策略
   * hdfs—site.xml设置复制因子，指定副本数量
* 机架策略
   * 集群放在不同的机架上，连在同一个交换机上，机架内带宽小
   * HDFS的机架感知
   * 本机架一个副本，别的一个副本
* 心跳机制
   * Namenode接收datanode的心跳和块报告
* 安全模式
   * Namenode在启动时会经过一个“安全模式“
* 校验和
   * 文件创立时都有checksum
   * 校验和会存储在一个隐藏文件夹里用于验证
* 回收站
   * 文件不会立马删除，可以快速恢复
* 元数据保护
   * 映像文件 + 事务日志是namenode的核心数据
   * 增加映像文件 + 事务日志的备份
* 快照

### HDFS文件操作
* 命令行方式
* API方式

###### 列出目录
* HDFS里面没有当前目录的概念，只能ls，不能cd
```
> bin/hadoop dfs -ls ./in/
```

###### 上传文件
* HDFS里面没有当前目录的概念，只能ls，不能cd
```
> bin/hadoop dfs -put path target_path
```

###### 复制到本地
```
> bin/hadoop dfs -get source_path output_path
```

###### 删除
```
> bin/hadoop dfs -rmr path
```

###### 查看内容
```
> bin/hadoop dfs -cat path
```

###### 查看多个文件
```
> zcat *.gz > abc       // 合并多个小文件到一个
``` 

###### 查看管理信息
```
> bin/hadoop dfsadmin -report
```

###### 安全模式
```
> bin/hadoop dfsadmin -safemode enter
```

#### Hadoop管理节点
##### 增加namenode
* 新机器安装好hadoop
* 复制namenode的配置文件复制到该节点
* 修改masters，slaves文件，增加该节点
* hadoop-daemon.sh start datanode/tasktracker
* 运行start-balancer.sh进行负载均衡

## Map Reduce
并行计算的框架

### 示例，气象局
* map: 每一条数据，map 提取年份操作，和对应气温
* shuffle：按年份聚合 (该步骤可选), shuffle 可以分摊 reduce阶段需要的计算量，一般用于reduce的机器数量很少
* reduce： 求每一年的最高气温 (也可以不需要reduce，直接写出数据就可以


* **mapper，reducer可以不同组合**
* **可以分治的算法就可以map-reduce**， map对应分，reduce
* reducer 的数量可以配置
* shuffler 在reduce之前对数据进行预处理和压缩， 减少了需要传输给reducer的**网络IO**
![HBASE Map Reduce](img/hadoop_map_reduce.png)

#### 代码例子
##### Mapper
```java
public static class TokenizerMapper extends Mapper<Object, Text, Text, IntWritable>{
    
    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();
    
    public void map(Object key, Text value, Context context) throws IOException, InterruptedException{
        System.out.println("key= " +  key.toString());
        System.out.println("value= " + value.toString());
    
        StringTokenizer itr = new StringTokenizer(value.toString());
        while(itr.hasMoreTokens()) {
            word.set(itr.nextToken());
            context.write(word, one);
        }
    }

}

```
* 继承java里的Mapper类，然后实现map方法

##### Reducer
```java
public static class IntSumReducer extends Reducer<Text, IntWritable, Text, IntWritable> {
    ...
    public void reduce(){};
}

```

##### 组合调用 main
```java
public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    String[] otherArgs = new GenericOptionsParser(conf, args).getRemainingArgs();
    
    Job job = new Job(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValeClass(IntWritable.class);
    
    job.waitForCompletion(true);
}
```
* 设置mapper，reducer，shuffer，output，提交作业

##### 性能调优
* 究竟需要多少reducer
* 输入：大文件由于小文件
* 减少网络IO: 压缩map的输出


#### 调度机制
* FIFO
* 支持公平调度
* 支持容量调度

### 工作机制
![HBASE Map Reduce](img/map_reduce_architecture.png)

#### 错误处理机制
##### 硬件故障
* 单点故障：jobtracker，namenode，选最牢靠的硬件
* jobtracker 通过 heartbeat(周期1min)监控task tracker。
* 如果没有发心跳 
    * 可能时硬件故障
    * 也可能负载过重，没来得及发
* 发现没有心跳：
    * 如果故障节点在执行map任务且没有完成，jobtracker会要求其他节点**重新执行**
    * 如果在执行reduce，尚未完成，jobtracker会要求其他节点**继续执行**
    
##### 软件故障，任务失败
* 代码缺陷引起的进程崩溃
* jvm自动退出，向tasktracker发送错误信息，错误信息写入日志
* tasktracker监听程序发现进程退出，标记任务失败
* 标记任务失败，把失败任务重新放入调度队列
* 如果失败超过阈值（4次），将不会被执行，任务宣告失败

### 运维
* 审计log

* 对hadoop集群进行监控
    * Ganglia
    * openstack
    * chukwa

##### 架构师
* 优缺点
* 瓶颈
* 使用场景
* 使用成本

## HBASE


## Pig
* pig 是hadoop的客户端
* 提供类似于SQL语句的面向数据流语言
* pig latin 可以进行排序，过滤，求和，分组，关联， 自定义函数。是一种面向数据分析处理的轻量级脚本语言
* pig 可以看作是 pig latin 到 map-reduce的映射器

## Hive
* 把SQL语句映射为mapReduce的查询


## YARN
* Yarn是一个资源调度平台，负责为运算程序提供服务器运算资源
* Yarn相当与一个**分布式操作系统平台**，map reduce等运算程序相当于应用程序

![HBASE Map Reduce](img/hadoop_yarn.png)

https://www.bilibili.com/video/av38472787?from=search&seid=8620089201747655937

#### hadoop 1.0
利用JobTracker框架来进行调度

* master/slave模式，利用jobTracker(master) 来调度taskTracker(slave) 来干活
* JobTracker：
    * 资源调度，任务监控（心跳）
* TaskTracker:
    * 监听自己机器的资源情况
    * 监视当前机器的tasks运行
    * 向JobTracker汇报
* 缺点：
    * 单点故障
    * JobTracker最多只支持4000节点的主机
    * 在TaskTracker端只是以简单的task数目来表示资源，而不是用cpu/内存的占用情况来表示
    * 在taskTracker端强行把资源划分为了map task，reduce task slot 浪费资源
    * JobTracker做的事情太多了，源代码复杂，增加维护成本
    * 只能运行MapReduce

#### hadoop 2.0
![HBASE Map Reduce](img/yarn.png)

* Resource Manager：
    * RM, 负责集群的自愿的统一管理和调度，
    * 处理客户端请求
    * 监控集群中的NM
* NodeManager：
    * NM, 负责自己所在节点的应用资源使用情况，并且向RM汇报
    * 接收并处理RM, AM的各种命令
* App Master: 比作项目经理，来进行具体的需要多少人，怎么分工， 分摊了之前JobTracker的很多工作
    * 每个应用程序对应一个AM, 负责应用程序管理
    * 向RM申请资源，并分给task
    * AM与NM通信，来启动或者停止task
* Container：
    * 封装了CPU,Memory等资源的容器
    * 相当于是一个任务运行环境的抽象
    * 网络io？
    
### 任务调度过程
##### FIFO 调度器

##### 容量调度器

##### 公平调度器


## Data serialization: Parquet, Avro

## 锁

#### paxos 算法 选举功能


### 大规模部署集群
https://www.bilibili.com/video/av23525072/?p=33

### 应用
https://www.bilibili.com/video/av23525072/?p=36
* 作为存储集群，有冗余备份
* map-reduce可以提供快速并行计算，可以进行数据分析

##### 场景 1： 日志分析
* CDN 加速技术，CDN服务器作为反向代理，还提供cache，分布在不同的物理节点，广州，上海 etc
* CDN 通过分析用户日志来cache
* 要排除日志里爬虫和程序的点击，e.g 用鼠标探测来反爬
* 跟踪用户，cookie

* 问题：
    * 日志的保存消耗空间
    * 日志需要备份
    * 统计效率低，时滞明显，不满足业务要求

###### 方案
* Hadoop + HBASE:
    * 设定数据定时过期，定期清理
    * 利用PIG 查询统计

#### 网络编程
* 链接
    * 长连接
    * 短连接
* 协议
    * tcp

#### 异步编程
* 多线程
* 多进程
* 锁